<a href="https://colab.research.google.com/github/nstsj/python_for_CL/blob/master/morphology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

установим пайморфи и майстем

In [1]:
!pip install pymystem==0.1.10
!pip install pymorphy2[fast]

ERROR: Could not find a version that satisfies the requirement pymystem==0.1.10 (from versions: none)
ERROR: No matching distribution found for pymystem==0.1.10
     |████████████████████████████████| 51kB 4.1MB/s 
     |████████████████████████████████| 7.1MB 18.1MB/s 
     |████████████████████████████████| 256kB 57.2MB/s 
  Created wheel for DAWG: filename=DAWG-0.7.8-cp36-cp36m-linux_x86_64.whl size=774178 sha256=e4af0c06cd354169309894c5d84bcd397fd2be95d29accbb7567a9ab4673200d
  Stored in directory: /root/.cache/pip/wheels/d4/88/d0/4e4abc83eb8f59a71e8dbd8ba99fd5615a3af1fac1ef7f8825
Successfully built DAWG


In [0]:
# pymorphy2[fast] - быстрая версия пайморфи. Если у вас windows, он вряд ли установится :(

In [3]:
# импорты
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

# сохраняем класс в переменную
mystem = Mystem() 
morph = MorphAnalyzer() 

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [0]:
# сэмпл-текст
text = """Сама система состоит из камеры и программного обеспечения, которое анализирует фотографию.
 Суть технологии — сопоставление лиц, попавших в объектив, с изображениями из базы данных."""

## нормализация

давайте приведем текст к нижнему регистру

In [11]:
text = text.lower()
text

'сама система состоит из камеры и программного обеспечения, которое анализирует фотографию.\n суть технологии — сопоставление лиц, попавших в объектив, с изображениями из базы данных.'

## токенизация

иногда текст нужно предварительно разбить на токены, отдельные элементы. Токенизаторов много, каждый делает это, руководствуясь своими правилами

In [12]:
# gensim
from gensim.utils import tokenize

list(tokenize(text, lowercase=True))[:10]

['сама',
 'система',
 'состоит',
 'из',
 'камеры',
 'и',
 'программного',
 'обеспечения',
 'которое',
 'анализирует']

In [0]:
# nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize, wordpunct_tokenize

In [14]:
wordpunct_tokenize(text)[:10]

['сама',
 'система',
 'состоит',
 'из',
 'камеры',
 'и',
 'программного',
 'обеспечения',
 ',',
 'которое']

In [16]:
word_tokenize(text)[:10]

['сама',
 'система',
 'состоит',
 'из',
 'камеры',
 'и',
 'программного',
 'обеспечения',
 ',',
 'которое']

## лемматизация и морфоанализ

### Mystem


Майстем работает немного лучше и сам токенизирует,
поэтому можно в него засовывать сырой текст.

In [0]:
# mystem.lemmatize функция лемматизации в майстеме
# сам объект mystem нужно заранее инициализировать
# мы сделали это в начале тетрадки строчкой "mystem = Mystem()"
mystem.lemmatize(text)[:10]

In [0]:
t = 'Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.'

In [0]:
# Если нужна грамматическая информация или надо сохранить ненормализованный текст,
# есть функция mystem.analyze
words_analized = mystem.analyze(text)

In [0]:
# возвращает она список словарей
# каждый словарь имеет либо одно поле 'text' (когда попался пробел) или text и analysis
# в analysis снова список словарей с вариантами разбора (первый самый вероятный)
# поля в analysis - 'gr' - грамматическая информация, 'lex' - лемма
# analysis - может быть пустым списком
words_analized[:10]

In [0]:
print('Слово - ', words_analized[1]['text'])
print('Разбор слова - ', words_analized[1]['analysis'][0])
print('Лемма слова - ', words_analized[1]['analysis'][0]['lex'])
print('Грамматическая информация слова - ', words_analized[1]['analysis'][0]['gr'])

In [0]:
#леммы можно достать в одну строчку
[parse['analysis'][0]['lex'] for parse in words_analized if parse.get('analysis')][:10]

Mystem умеет разбивать текст на предложения, но через питоновский интерфейс это сделать не получится. Нужно скачать mystem отсюда - https://yandex.ru/dev/mystem/

После этого сохранить текст в файл.

In [0]:
f = open('text.txt', 'w')
f.write(text)
f.close()

Из командной строки или из питона запустить майстем на нашем файле

In [0]:
# про параметры почитайте в ./mystem -h
os.system(' ./mystem -iscd --format json text.txt text_parsed.txt')

В целевом файле теперь лежит разобранный текст в jsonlines (json на каждой строчке)

In [0]:
t = [json.loads(line) for line in open('text_parsed.txt')]

Каждый объект в этом списке - параграф. Каждый параграф на предложения можно разбив по тегу '//s'

Ещё так вызывать майстем может понадобиться, если важна скорость.

Недостатки Mystem: это продукт Яндекса с некоторыми ограничениями на использование, больше он не развивается.

Важным достоинством Mystem является то, что он работает не с отдельными словами, а с целым предложением. При определении нужной леммы учитывается контекст, что позволяет во многих случаях разрешать омонимию.

### Pymorphy

Pymorphy - открытый и развивается (можно поучаствовать на гитхабе)

Ссылка на репозиторий: https://github.com/kmike/pymorphy2

Попробуйте сразу установить быструю версию (pip install pymorphy2[fast])

У него нет втстроенной токенизации и он расценивает всё как слово. Когда есть несколько вариантов, он выдает их с вероятностостями, которые расчитатны на корпусе со снятой неоднозначностью. Это лучше стемминга, но хуже майстема.

In [0]:
# основная функция - pymorphy.parse
words_analized = [morph.parse(token) for token in word_tokenize(text)]

In [0]:
morph.parse("печь")

In [0]:
# Она похожа на analyze в майстеме только возрващает список объектов Parse
# Первый в списке - самый вероятный разбор (у каждого есть score)
# Информация достается через атрибут (Parse.word - например)
# Грамматическая информация хранится в объекте OpencorporaTag и из него удобно доставать
# части речи или другие категории
print('Первое слово - ', words_analized[0][0].word)
print('Разбор первого слова - ', words_analized[0][0])
print('Лемма первого слова - ', words_analized[0][0].normal_form)
print('Грамматическая информация первого слова - ', words_analized[0][0].tag)
print('Часть речи первого слова - ', words_analized[0][0].tag.POS)
print('Род первого слова - ', words_analized[0][0].tag.gender)
print('Число первого слова - ', words_analized[0][0].tag.number)
print('Падеж первого слова - ', words_analized[0][0].tag.case)